In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df=pd.read_excel("C:/Users/adity/A2 ML/Train.xlsx")
df['y'] = df['y'].replace({'yes': 1, 'no': 0})
# df = df.drop('day', axis=1)

# day had barely any correlation with y and would have caused overfitting. However it was later clarified that we can't drop features

# balance_mean = df['balance'].mean()
# balance_std = df['balance'].std()

# duration_mean = df['duration'].mean()
# duration_std = df['duration'].std()

# df['balance'] = (df['balance'] - balance_mean) / balance_std
# df['duration'] = (df['duration'] - duration_mean) / duration_std

#scaling just took extra time and provided no benefit



In [2]:
import pandas as pd

features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome','month']

for feature in features:
    print(f"Counts for feature: {feature}")
    counts = df.groupby([feature, 'y']).size().unstack(fill_value=0)
    counts.columns = ['y=0', 'y=1']
    counts['ratio']=counts['y=0']/(counts['y=0']+counts['y=1'])
    print(counts)
    print("\n")


Counts for feature: job
                y=0   y=1     ratio
job                                
admin.         4540   631  0.877973
blue-collar    9024   708  0.927250
entrepreneur   1364   123  0.917283
housemaid      1131   109  0.912097
management     8157  1301  0.862444
retired        1748   516  0.772085
self-employed  1392   187  0.881571
services       3785   369  0.911170
student         669   269  0.713220
technician     6757   840  0.889430
unemployed     1101   202  0.844973
unknown         254    34  0.881944


Counts for feature: marital
            y=0   y=1     ratio
marital                        
divorced   4585   622  0.880545
married   24459  2755  0.898765
single    10878  1912  0.850508


Counts for feature: education
             y=0   y=1     ratio
education                       
primary     6260   591  0.913735
secondary  20752  2450  0.894406
tertiary   11305  1996  0.849936
unknown     1605   252  0.864297


Counts for feature: default
           y=0   y=1  

In [3]:

# Job group mapping
job_group_mapping = {
    'admin.': 'white-collar',
    'management': 'white-collar',
    'technician': 'white-collar',
    'blue-collar': 'Blue-collar',
    'services': 'Blue-collar',
    'housemaid': 'Blue-collar',
    'entrepreneur': 'Self-employed',
    'self-employed': 'Self-employed',
    'student': 'Student',
    'unemployed': 'Unemployed',
    'unknown': 'Unknown',
    'retired': 'Retired'
}

df['job_group'] = df['job'].map(job_group_mapping)

# Month group mapping
month_group_mapping = {
    'jan': 'jan-feb',
    'feb': 'jan-feb',
    'mar': 'mar',
    'apr': 'apr',
    'may': 'may-aug',  
    'jun': 'may-aug',  
    'jul': 'may-aug',  
    'aug': 'may-aug',  
    'sep': 'sep-oct',
    'oct': 'sep-oct',
    'nov': 'nov',
    'dec': 'dec'
}

df['month_group'] = df['month'].map(month_group_mapping)

# Drop the original 'job' and 'month' columns
df = df.drop('job', axis=1)
df = df.drop('month', axis=1)

y_column = df.pop('y')
df['y'] = y_column

print(df.head())




   age  marital  education default  balance housing loan  contact  day  \
0   58  married   tertiary      no     2143     yes   no  unknown    5   
1   44   single  secondary      no       29     yes   no  unknown    5   
2   33  married  secondary      no        2     yes  yes  unknown    5   
3   47  married    unknown      no     1506     yes   no  unknown    5   
4   33   single    unknown      no        1      no   no  unknown    5   

   duration  campaign  pdays  previous poutcome      job_group month_group  y  
0       261         1     -1         0  unknown   white-collar     may-aug  0  
1       151         1     -1         0  unknown   white-collar     may-aug  0  
2        76         1     -1         0  unknown  Self-employed     may-aug  0  
3        92         1     -1         0  unknown    Blue-collar     may-aug  0  
4       198         1     -1         0  unknown        Unknown     may-aug  0  


In [4]:
unique_counts = df.nunique()
print(unique_counts)

def most_common_value(y):
    count_dict = {}
    for val in y:
        if val in count_dict:
            count_dict[val] += 1
        else:
            count_dict[val] = 1

    most_common = None
    max_count = -1
    
    for key, count in count_dict.items():
        if count > max_count:
            most_common = key
            max_count = count
    
    return most_common



age              77
marital           3
education         4
default           2
balance        7168
housing           2
loan              2
contact           3
day              31
duration       1573
campaign         48
pdays           559
previous         41
poutcome          4
job_group         7
month_group       7
y                 2
dtype: int64


In [5]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None, n_samples=None, impurity=None):
        self.feature = feature
        self.threshold = threshold  # number in case of numerical, string in case of categorical
        self.left = left
        self.right = right
        self.value = value  # Only set for leaf nodes
        self.n_samples = n_samples  
        self.impurity= impurity  # for leaf nodes  
    
    def is_leaf_node(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=10, min_gain=0, criterion="gini", _class_weight=1.0):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.min_gain = min_gain
        self.criterion = criterion  # "gini" or "entropy"
        self.root = None
        self._class_weight = _class_weight  # Weight of y=1 relative to y=0
        
    def fit(self, X, y):
        self.root = self._grow_tree(X, y)
        
    def _grow_tree(self, X, y, depth=0):
        n_samples = X.shape[0]
        n_labels = len(np.unique(y))
        
        # Stopping criteria
        if depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
            value = most_common_value(y)
            if self.criterion == "gini":
                Impurity = self._gini(y)  
            else:
                Impurity = self._entropy(y)  
            
            # Return a leaf node with the calculated impurity
            return Node(value=value, n_samples=n_samples, impurity=Impurity)
        
        best_feature, best_threshold, best_gain = self._best_split(X, y)
        
        # Stop if no split is good enough
        if best_gain < self.min_gain:
            value = most_common_value(y)
            
            if self.criterion == "gini":
                Impurity = self._gini(y)
            else:
                Impurity = self._entropy(y)
            
            return Node(value=value, n_samples=n_samples, impurity=Impurity)
        
        left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
        
        # Recurse
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        
        return Node(feature=best_feature, threshold=best_threshold, left=left, right=right, n_samples=n_samples)
        
    def _best_split(self, X, y):
        best_gain = -1
        split_idx, split_threshold = None, None

        n_feats = X.shape[1]  # total number of features

        for feat_idx in range(n_feats):
            X_column = X[:, feat_idx]
            thresholds = None

            # For numerical features, only check thresholds where y changes
            if feat_idx in ['duration', 'pdays', 'balance']: 
                # Sort X_column and y based on the values in X_column
                sorted_idx = np.argsort(X_column)
                X_column_sorted, y_sorted = X_column[sorted_idx], y[sorted_idx]

                # Identify points where the label of y changes
                thresholds = []
                for i in range(len(y_sorted) - 1):
                    if y_sorted[i] != y_sorted[i + 1]:
                        # Add the midpoint between two consecutive points where y changes
                        thresholds.append((X_column_sorted[i] + X_column_sorted[i + 1]) / 2)
            else:
                # Otherwise, check for all unique values
                thresholds = np.unique(X_column)

            # Iterate through the identified thresholds
            for thr in thresholds:
                # Calculate information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold, best_gain

    
    def _information_gain(self, y, X_column, threshold):
        if self.criterion == "gini":
            parent_impurity = self._gini(y)
        else:
            parent_impurity = self._entropy(y)
        
        left_idxs, right_idxs = self._split(X_column, threshold)
        
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        n = len(y)
        n_l, n_r = len(left_idxs) / n, len(right_idxs) / n
        
        # Weighted average of the child impurities
        if self.criterion == "gini":
            return parent_impurity - (n_l * self._gini(y[left_idxs]) + n_r * self._gini(y[right_idxs]))
        else:
            return parent_impurity - (n_l * self._entropy(y[left_idxs]) + n_r * self._entropy(y[right_idxs]))
        
    def _split(self, X_column, split_thresh):
        # Check if the feature is numerical (dtype is float or int)
        if np.issubdtype(X_column.dtype, np.number):
            # For numerical features
            left_idx = np.argwhere(X_column <= split_thresh).flatten()
            right_idx = np.argwhere(X_column > split_thresh).flatten()
        else:
            # For categorical features
            left_idx = np.argwhere(X_column == split_thresh).flatten()
            right_idx = np.argwhere(X_column != split_thresh).flatten()

        return left_idx, right_idx

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)

        # Ensure we have both class labels (y=0 and y=1) in ps
        if len(ps) > 1:
            # Apply class weight to y=1 if it exists
            ps_weighted = np.copy(ps)
            ps_weighted[1] *= self._class_weight
        else:
            # If there's only one class, no need to weight
            ps_weighted = ps

        return -np.sum([p * np.log2(p) for p in ps_weighted if p > 0])

    
    def _gini(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)

        if len(ps) > 1:
            # Apply class weight to y=1 if it exists
            ps_weighted = np.copy(ps)
            ps_weighted[1] *= self._class_weight
        else:
            # If there's only one class, no need to weight
            ps_weighted = ps

        return 1 - np.sum([p ** 2 for p in ps_weighted])

        
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def _traverse_tree(self, X, node):
        if node.is_leaf_node():
            return node.value
        
        feature_value = X[node.feature]
        
        if isinstance(feature_value, (int, float)):
            # Numerical feature: go to left if X[feature] <= threshold, else go right
            if feature_value <= node.threshold:
                return self._traverse_tree(X, node.left)
            else:
                return self._traverse_tree(X, node.right)
        else:
            # Categorical feature: go to left if X[feature] == threshold, else go right
            if feature_value == node.threshold:
                return self._traverse_tree(X, node.left)
            else:
                return self._traverse_tree(X, node.right)



    def get_max_depth(self):
        def _depth(node):
            if node.is_leaf_node():
                return 0
            return 1 + max(_depth(node.left), _depth(node.right))

        return _depth(self.root)

    def _count_leaves(self,node):
        if node.is_leaf_node():
            return 1
        return self._count_leaves(node.left) + self._count_leaves(node.right)

    def get_num_leaves(self):
        return self._count_leaves(self.root)
    
    def _subtree_risk(self, node):
        if node.is_leaf_node():
            # The risk of a leaf node is its impurity weighted by its number of samples
            return node.impurity * node.n_samples
        
        return self._subtree_risk(node.left) + self._subtree_risk(node.right)

    def cost_complexity(self, node, alpha):
        """
        Cost Complexity = Impurity (Risk) + alpha * Number of Leaves
        """
        # Calculate the risk (impurity) of the subtree
        risk = self._subtree_risk(node)
        
        # Count the number of leaves in the subtree
        n_leaves = self._count_leaves(node)
        
        # Return the total cost complexity: risk + alpha * number of leaves
        return risk + alpha * n_leaves



In [6]:


def Precision(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))

    return tp / (tp + np.sum((y_pred == 1) & (y_true == 0)))

def Recall(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))

    return tp / (tp + np.sum((y_true == 1) & (y_pred == 0)))

def F1_score(y_true, y_pred):
    p = Precision(y_true, y_pred)
    r = Recall(y_true, y_pred)

    return 2 * (p * r) / (p + r)

def Accuracy(y_true, y_pred): 
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    return (tp + tn) / (tp + tn + fp + fn)

def train_test_split(X, y, test_size=0.2, random_state=None):
    # Set random seed for reproducibility
    if random_state is not None:
        np.random.seed(random_state)
    
    n_samples = X.shape[0]

    indices = np.arange(n_samples)  
    np.random.shuffle(indices)      
    
    X_shuffled = X[indices]
    y_shuffled = y[indices]
    test_size_samples = int(test_size * n_samples)
    
    X_train = X_shuffled[:-test_size_samples]  
    X_cv = X_shuffled[-test_size_samples:]
    
    y_train = y_shuffled[:-test_size_samples] 
    y_cv = y_shuffled[-test_size_samples:]    
    
    return X_train, X_cv, y_train, y_cv

X = df[df.columns[:-1]].values
y = df[df.columns[-1]].values

X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.3, random_state=24)  
X_cv, X_test, y_cv, y_test = train_test_split(X_cv, y_cv, test_size=0.67, random_state=42)

# ros = RandomUnderSampler(random_state=42)
# X_train_oversampled, y_train_oversampled = ros.fit_resample(X_train, y_train)
# didn't produce good results, also SMOTE could not be applied to categorical features directly, so I just used class weight

tree = DecisionTree(min_samples_split=19, max_depth=20,min_gain=0.02, criterion="gini", _class_weight=2.4)
# best set of parameters: 19,20,0.02,gini,2.4

tree.fit(X_train, y_train)
y_train_pred = tree.predict(X_train)
y_pred = tree.predict(X_cv)

train_accuracy=Accuracy(y_train, y_train_pred)
train_f1= F1_score(y_train, y_train_pred)
train_precision=Precision(y_train, y_train_pred)
train_recall=Recall(y_train, y_train_pred)

print("Training Accuracy: ", train_accuracy)
print("Training F1 Score: ", train_f1)  
print("Training Precision: ", train_precision)
print("Training Recall: ", train_recall)

cv_accuracy = Accuracy(y_cv, y_pred)
cv_f1 = F1_score(y_cv, y_pred)
cv_precision=Precision(y_cv, y_pred)
cv_recall=Recall(y_cv, y_pred)

print(f"CV Accuracy: {cv_accuracy}")
print(f"CV F1 Score: {cv_f1}")
print(f"CV Precision: {cv_precision}")
print(f"CV Recall: {cv_recall}")

# cost_complexity_value = tree.cost_complexity(tree.root, alpha=0.01)
# print("Cost Complexity of the tree:", cost_complexity_value)

max_depth = tree.get_max_depth()
num_leaves = tree.get_num_leaves()

print(f"Max Depth of the Tree: {max_depth}")
print(f"Number of Leaf Nodes: {num_leaves}")




Training Accuracy:  0.8934845803842265
Training F1 Score:  0.2823078560783479
Training Precision:  0.6597014925373135
Training Recall:  0.1795774647887324
CV Accuracy: 0.8873994638069705
CV F1 Score: 0.24776119402985075
CV Precision: 0.6148148148148148
CV Recall: 0.15514018691588785
Max Depth of the Tree: 20
Number of Leaf Nodes: 54


In [7]:


# X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.3, random_state=24)

# min_samples_split_values = np.arange(5, 40, 4)  
# max_depth_values = np.arange(16, 25, 2)         
# min_gain_values = np.arange(0.005, 0.036, 0.005) 

# best_f1_score = -1
# best_params = None

# # Perform grid search
# for min_samples_split in min_samples_split_values:
#     for max_depth in max_depth_values:
#         for min_gain in min_gain_values:
#             # Initialize the DecisionTree with current hyperparameters
#             tree = DecisionTree(min_samples_split=min_samples_split,
#                                 max_depth=max_depth,
#                                 min_gain=min_gain,
#                                 criterion="gini", 
#                                 _class_weight=2.4)

#             tree.fit(X_train, y_train)
#             y_cv_pred = tree.predict(X_cv)
#             f1 = F1_score(y_cv, y_cv_pred)
            
#             if f1 > best_f1_score:
#                 best_f1_score = f1
#                 best_params = {
#                     'min_samples_split': min_samples_split,
#                     'max_depth': max_depth,
#                     'min_gain': min_gain
#                 }

# # Print the best hyperparameters and F1 score
# print(f"Best F1 Score: {best_f1_score}")
# print(f"Best Parameters: {best_params}")


In [8]:

def check(node, X_node, y_node, tree, X_cv, y_cv, curr_recall, delta=0.125):
    
    if node.is_leaf_node() :
        return
    
    left_idxs, right_idxs = tree._split(X_node[:, node.feature], node.threshold)

    X_left, y_left = X_node[left_idxs], y_node[left_idxs]
    X_right, y_right = X_node[right_idxs], y_node[right_idxs]
    
    check(node.left, X_left, y_left, tree, X_cv, y_cv, curr_recall, delta)
    check(node.right, X_right, y_right, tree, X_cv, y_cv, curr_recall, delta)

    if node.left.is_leaf_node() and node.right.is_leaf_node():  
        # Temporarily prune this node
        node.value = most_common_value(y_node)

        y_pred_temp = tree.predict(X_cv)
        temp_recall = Recall(y_cv, y_pred_temp)

        if temp_recall < (1 - delta) * curr_recall:
            # If pruning worsens recall significantly, restore it
            node.value= None
        else:
            curr_recall = temp_recall
            
check(tree.root, X_train, y_train, tree, X_cv, y_cv, cv_recall, 0)

max_depth = tree.get_max_depth()
num_leaves = tree.get_num_leaves()

print(f"Max Depth of the Tree: {max_depth}")
print(f"Number of Leaf Nodes: {num_leaves}")

y_test_pred = tree.predict(X_test)

test_accuracy = Accuracy(y_test, y_test_pred)
test_f1 = F1_score(y_test, y_test_pred)
test_precision = Precision(y_test, y_test_pred)
test_recall = Recall(y_test, y_test_pred)

print("Test Accuracy: ", test_accuracy)
print("Test F1 Score: ", test_f1)
print("Test Precision: ", test_precision)
print("Test Recall: ", test_recall)


Max Depth of the Tree: 7
Number of Leaf Nodes: 8
Test Accuracy:  0.8924837680202488
Test F1 Score:  0.2945848375451263
Test Precision:  0.631578947368421
Test Recall:  0.192090395480226


In [11]:
df_test=pd.read_excel("C:/Users/adity/A2 ML/Test.xlsx")


# Job group mapping
job_group_mapping = {
    'admin.': 'white-collar',
    'management': 'white-collar',
    'technician': 'white-collar',
    'blue-collar': 'Blue-collar',
    'services': 'Blue-collar',
    'housemaid': 'Blue-collar',
    'entrepreneur': 'Self-employed',
    'self-employed': 'Self-employed',
    'student': 'Student',
    'unemployed': 'Unemployed',
    'unknown': 'Unknown',
    'retired': 'Retired'
}

df_test['job_group'] = df_test['job'].map(job_group_mapping)

# Month group mapping
month_group_mapping = {
    'jan': 'jan-feb',
    'feb': 'jan-feb',
    'mar': 'mar',
    'apr': 'apr',
    'may': 'may-aug',  
    'jun': 'may-aug',  
    'jul': 'may-aug',  
    'aug': 'may-aug',  
    'sep': 'sep-oct',
    'oct': 'sep-oct',
    'nov': 'nov',
    'dec': 'dec'
}

df_test['month_group'] = df_test['month'].map(month_group_mapping)

# Drop the original 'job' and 'month' columns
df_test = df_test.drop('job', axis=1)
df_test = df_test.drop('month', axis=1)

   id  age  marital  education default  balance housing loan   contact  day  \
0   1   30  married    primary      no     1787      no   no  cellular   19   
1   2   33  married  secondary      no     4789     yes  yes  cellular   11   
2   3   35   single   tertiary      no     1350     yes   no  cellular   16   
3   4   30  married   tertiary      no     1476     yes  yes   unknown    3   
4   5   59  married  secondary      no        0     yes   no   unknown    5   

   duration  campaign  pdays  previous poutcome     job_group month_group  
0        79         1     -1         0  unknown    Unemployed     sep-oct  
1       220         1    339         4  failure   Blue-collar     may-aug  
2       185         1    330         1  failure  white-collar         apr  
3       199         4     -1         0  unknown  white-collar     may-aug  
4       226         1     -1         0  unknown   Blue-collar     may-aug  


In [13]:
X_test = df[df.columns[:]].values
y_test_pred=tree.predict(X_test)
print(y_test_pred)

[0 0 0 ... 1 0 0]
